# USING ML using area and spikes on SSVEP

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

In [62]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [63]:
from sklearn.preprocessing import MinMaxScaler

def preprocess(t,label, fileName):
    
    label1 = 8
    label2 = 14
    tolr = 1.5
    
    data = np.loadtxt(fileName,delimiter=',' , skiprows=1)
    data = data[:,[22]]
    arr = []
    
    y = data[:,0][256: t*256]

    y = butter_highpass_filter(y,5,256,5)
    ps = np.abs(np.fft.fft(y))**2
    
    arr.append(ps)
        
    freqs = np.fft.fftfreq( y.size , float(1)/256 )
    arr.append(freqs)
    
    output_arr = np.array(arr)
    output_arr = np.transpose(output_arr)
    
    # DROP WHEN FREQUENCY < 0
    output_arr = output_arr[ np.logical_not( output_arr[:,1] < 0 ) ]
    
    rows,cols = output_arr.shape
    
    total1 = 0
    total2 = 0
    max1 = 0
    max2 = 0
    
    for x in range (0,rows):
        if ((output_arr[x][1] <= label1 + tolr) and (output_arr[x][1] >= label1 - tolr)):
            total1 += output_arr[x][0]
            if (max1 < output_arr[x][0]):
                max1 = output_arr[x][0]
        if ((output_arr[x][1] <= label2 + tolr) and (output_arr[x][1] >= label2 - tolr)):
            total2 += output_arr[x][0]
            if (max2 < output_arr[x][0]):
                max2 = output_arr[x][0]

  
    fin_arr = np.array([[max1,max2,label]])
    
    return fin_arr

In [64]:
preprocess(2,8, '../Data/SSVEP_Public/SSVEP_8Hz_Trial1_SUBJ3.MAT.csv')

array([[  4.72758972e+05,   2.88292474e+05,   8.00000000e+00]])

In [65]:
eight_ssvep =   ['../Data/SSVEP_Public/SSVEP_8Hz_Trial1_SUBJ1.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial1_SUBJ2.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial1_SUBJ3.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial1_SUBJ4.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial2_SUBJ1.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial2_SUBJ2.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial2_SUBJ3.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial2_SUBJ4.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial3_SUBJ1.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial3_SUBJ2.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial3_SUBJ3.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial3_SUBJ4.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial4_SUBJ1.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial4_SUBJ2.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial4_SUBJ3.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_8Hz_Trial4_SUBJ4.MAT.csv',]

fourteen_ssvep =   ['../Data/SSVEP_Public/SSVEP_14Hz_Trial1_SUBJ1.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial1_SUBJ2.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial1_SUBJ3.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial1_SUBJ4.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial2_SUBJ1.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial2_SUBJ2.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial2_SUBJ3.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial2_SUBJ4.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial3_SUBJ1.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial3_SUBJ2.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial3_SUBJ3.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial3_SUBJ4.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial4_SUBJ1.MAT.csv',
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial4_SUBJ2.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial4_SUBJ3.MAT.csv', 
                 '../Data/SSVEP_Public/SSVEP_14Hz_Trial4_SUBJ4.MAT.csv',]

In [66]:
time = 5
train_data = np.concatenate((   preprocess(time,8, eight_ssvep[0]),    
                                preprocess(time,8, eight_ssvep[1]),    
                                preprocess(time,8, eight_ssvep[2]),    
                                preprocess(time,8, eight_ssvep[12]), 
                                preprocess(time,14, fourteen_ssvep[0]),  
                                preprocess(time,14, fourteen_ssvep[1]),  
                                preprocess(time,14, fourteen_ssvep[2]),  
                                preprocess(time,14, fourteen_ssvep[12]),
                                preprocess(time,8, eight_ssvep[4]),    
                                preprocess(time,8, eight_ssvep[5]),    
                                preprocess(time,8, eight_ssvep[6]),    
                                preprocess(time,8, eight_ssvep[13]), 
                                preprocess(time,14, fourteen_ssvep[4]),  
                                preprocess(time,14, fourteen_ssvep[5]),  
                                preprocess(time,14, fourteen_ssvep[6]),  
                                preprocess(time,14, fourteen_ssvep[13]),
                                preprocess(time,8, eight_ssvep[8]),    
                                preprocess(time,8, eight_ssvep[9]),    
                                preprocess(time,8, eight_ssvep[10]),    
                                preprocess(time,8, eight_ssvep[14]), 
                                preprocess(time,14, fourteen_ssvep[8]),  
                                preprocess(time,14, fourteen_ssvep[9]),  
                                preprocess(time,14, fourteen_ssvep[10]),  
                                preprocess(time,14, fourteen_ssvep[14]),
                                preprocess(time,8, eight_ssvep[3]),    
                                preprocess(time,8, eight_ssvep[7]),    
                                preprocess(time,8, eight_ssvep[11]),    
                                preprocess(time,8, eight_ssvep[15]), 
                                preprocess(time,14, fourteen_ssvep[3]),  
                                preprocess(time,14, fourteen_ssvep[7]),  
                                preprocess(time,14, fourteen_ssvep[11]),  
                                preprocess(time,14, fourteen_ssvep[15])
                            ), axis=0)

In [67]:
print train_data.shape

(32, 3)


In [68]:
train_X = train_data[:,:-1]
train_Y = train_data[:,-1]

In [69]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix

classifiers = [ 
                    DecisionTreeClassifier(),
                    KNeighborsClassifier(n_neighbors=9),
                    GaussianNB(),
                    NearestCentroid(),
                    RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs = -1)
                  ]

In [70]:
for clf in classifiers:
        print ""
        print clf
 
        predicted = cross_val_predict(clf, train_X, train_Y, cv=10)
        
        print(confusion_matrix(train_Y, predicted))
        print("Cross-validation accuracy: ", metrics.accuracy_score(train_Y, predicted))


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
[[10  6]
 [ 8  8]]
('Cross-validation accuracy: ', 0.5625)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='uniform')
[[ 9  7]
 [ 6 10]]
('Cross-validation accuracy: ', 0.59375)

GaussianNB(priors=None)
[[ 3 13]
 [ 3 13]]
('Cross-validation accuracy: ', 0.5)

NearestCentroid(metric='euclidean', shrink_threshold=None)
[[ 3 13]
 [ 2 14]]
('Cross-validation accuracy: ', 0.53125)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=